In [ ]:
import mitsuba as mi
import drjit as dr
import numpy as np
import matplotlib.pyplot as plt
from sionna.rt import load_scene, RadioMapSolver
import inspect

from scene_objects import create_scene_objects, gather_bboxes, visualize_scene_collisions
from motion_engine import MotionEngine

# Set Mitsuba variant
mi.set_variant("llvm_ad_mono_polarized")

In [ ]:
# Get all functions in the class
for name, func in inspect.getmembers(MotionEngine, predicate=inspect.isfunction):
    sig = inspect.signature(func)
    print(f"{name}{sig}")

In [ ]:
scene_path = r"C:\Users\sawyer\Documents\sionna_rt_jamming\data\testy\DowntownChicago1KM.xml"
scene = load_scene(scene_path)

In [ ]:
bounds, map_center, map_size, cell_size = create_scene_objects(scene)
map_width, map_height = map_size

In [ ]:
engine = MotionEngine(scene, bounds=bounds)

# Add jammer paths
engine.add_random_walk("Tx1", starting_position=[70, -10, 10], num_steps=3, step_size=10.0, time_step=1.0)
engine.add_random_walk("Tx2", starting_position=[-260, 100, 10], num_steps=5, step_size=10.0, time_step=1.0)

# Create synchronized padded paths
padded_paths, max_steps = engine.create_path_matrix(padding_mode='pad_end')

In [ ]:
rm_solver = RadioMapSolver()

for step in range(max_steps):
    # Update transmitter positions
    engine.update_scene_transmitters(step)
    
    # Print current positions
    positions = engine.get_all_positions_at_step(step)
    for jammer_id, pos in positions.items():
        print(f"{jammer_id} position: [{pos[0]:.1f}, {pos[1]:.1f}, {pos[2]:.1f}]")
    
    # Compute radio map
    rm = rm_solver(
        scene,
        max_depth=10,
        samples_per_tx=10**6,  # Adjust for speed/memory
        cell_size=cell_size,
        center=map_center,
        size=[map_width, map_height],
        orientation=[0, 0, 0],
        diffraction=True,
        refraction=True,
        edge_diffraction=True
    )
    
    # Display radio map
    rm.show(metric="rss", tx=None, show_tx=True, vmin=-200)

In [ ]:
print(padded_paths)

In [ ]:
building_data = gather_bboxes(r"C:\Users\sawyer\Documents\sionna_rt_jamming\data\testy\meshes")
visualize_scene_collisions(building_data, engine.jammer_paths)